## Importing necessary libraries

In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

## Importing the dataset 'bank.csv'

In [2]:
dataset = pd.read_csv('bank.csv', delimiter=';')
dataset=dataset.drop(['job','education','contact','housing', 'poutcome','loan','pdays','campaign','day','month','y','previous'], axis=1)
# dataset

## Filtering and encoding the dataset

In [3]:
obj_df=dataset.select_dtypes(include=['object']).copy()
int_df=dataset.select_dtypes(include=['int64']).copy()
# obj_df
# int_df
# dataset.dtypes

In [4]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
for i in list(obj_df):
    obj_df[i] = ord_enc.fit_transform(obj_df[[i]])
    obj_df[i] = obj_df[i].astype(int)
# obj_df

## Final dataset

In [5]:
X = pd.concat([obj_df, int_df], axis=1, join="inner")
# X=X.sample(n=2000)
X

,marital,default,age,balance,duration
0,1,0,30,1787,79
1,1,0,33,4789,220
2,2,0,35,1350,185
3,1,0,30,1476,199
4,1,0,59,0,226
...,...,...,...,...,...
4516,1,0,33,-333,329
4517,1,1,57,-3313,153
4518,1,0,57,295,151
4519,1,0,28,1137,129


In [6]:
print('Total number of points =', len(X))

Total number of points = 4521


## Dataset used for vanilla clustering

In [7]:
Vanilla_data = X.drop(['marital','default'], axis=1)
Vanilla_data_copy=Vanilla_data

## Vanilla KMeans++ Clustering 

In [8]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 0)
y_kmeans = kmeans.fit_predict(Vanilla_data)
S=list(kmeans.cluster_centers_)
print('Cost of vanilla clustering = ', kmeans.inertia_)
print('Centers of the clusters: ',S)

Cost of vanilla clustering =  6612255596.944933
Centers of the clusters:  [array([ 40.7614927 , 454.18334235, 263.51189832]), array([   43.09090909, 10833.53146853,   233.83916084]), array([  42.86605784, 3952.71841705,  276.23896499]), array([   46.47826087, 26353.69565217,   172.7826087 ])]


In [9]:
# X.index[X['marital']==1.0].tolist()

## Protected groups (marital and default)

In [10]:
C_married={i:[] for i in set(X['marital'].values)}

for i in range(len(X['marital'])) :
    C_married[X['marital'].iat[i]].append(X.index[i]) 
    
C_ed = {i:[] for i in set(X['default'].values)}

for i in range(len(X['default'])) :
    C_ed[X['default'].iat[i]].append(X.index[i]) 

## MP and RD values for fairness bounds (alpha and beta)

In [11]:
alpha_married = {0: 0.75, 1: 0.75, 2: 0.75}
beta_married = {0: 0.1, 1: 0.15, 2: 0.15}
alpha_ed = {0: 0.99, 1: 0.99}
beta_ed = {0: 0.05, 1: 0.01}

# list(C_married.keys())
# print(S[0])

## LP1

In [12]:
model=Model("LP1")

x={}
for v in list(X.index):
    for f in range(len(S)):
        x[v,f]=model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1)
        
for v in list(X.index):
    model.addConstr(quicksum(x[v,f] for f in range(len(S)))==1)

for f in range(len(S)):
    for i in list(C_married.keys()):
        model.addConstr(quicksum(x[v,f] for v in C_married[i]) <= alpha_married[i]*quicksum(x[v,f] for v in list(X.index)))
        model.addConstr(quicksum(x[v,f] for v in C_married[i]) >= beta_married[i]*quicksum(x[v,f] for v in list(X.index)))

for f in range(len(S)):
    for i in list(C_ed.keys()):
        model.addConstr(quicksum(x[v,f] for v in C_ed[i]) <= alpha_ed[i]*quicksum(x[v,f] for v in list(X.index)))
        model.addConstr(quicksum(x[v,f] for v in C_ed[i]) >= beta_ed[i]*quicksum(x[v,f] for v in list(X.index)))
        
Obj=0
for v in list(Vanilla_data.index):
    for f in range(len(S)):
        V = (np.array(Vanilla_data[Vanilla_data.index==v])[0])

        Obj+=x[v,f]*(np.linalg.norm(V-np.array(S[f]))**2)
    
model.setObjective(Obj)
model.optimize()

Academic license - for non-commercial use only - expires 2022-08-26
Using license file C:\Users\akgo1\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4561 rows, 18084 columns and 198924 nonzeros
Model fingerprint: 0x7f416609
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e+01, 5e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 4 rows and 0 columns
Presolve time: 0.22s
Presolved: 4557 rows, 18084 columns, 180840 nonzeros

Ordering time: 0.02s

Barrier statistics:
 AA' NZ     : 1.629e+05
 Factor NZ  : 2.203e+05 (roughly 10 MBytes of memory)
 Factor Ops : 1.091e+07 (less than 1 second per iteration)
 Threads    : 3

Ba

## Assigning v with x*[v,f]=1 to final_asst, the dictionary containing the final fair clustered points

In [13]:
x_star={}
for v in list(Vanilla_data.index):
    for f in range(len(S)):
        x_star[v,f]=x[v,f].X
        
final_asst = {f: [] for f in range(len(S))}

for v in list(Vanilla_data.index):
    for f in range(len(S)):
        if x_star[v,f]>=1:
            Vanilla_data=Vanilla_data.drop(v)
            final_asst[f].append(v)
# len(Vanilla_data)
final_asst

{0: [0,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  11,
  12,
  13,
  14,
  15,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  26,
  27,
  28,
  29,
  31,
  32,
  34,
  35,
  36,
  37,
  39,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  50,
  51,
  52,
  53,
  54,
  56,
  57,
  58,
  59,
  60,
  61,
  63,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  73,
  74,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  95,
  97,
  99,
  100,
  101,
  104,
  105,
  106,
  107,
  109,
  111,
  112,
  113,
  114,
  115,
  116,
  118,
  119,
  120,
  121,
  123,
  124,
  125,
  126,
  127,
  128,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  171,
  172,
  173,
  175,
  177,
  178,
  179,
  180,
  181,
  183,
  184,
  185,
  186,
  187,
  

In [14]:
# for v in list(Vanilla_data.index):
#     for f in range(len(S)):
#         print(x_star[v,f])

## Checking the number of variables in final_asst

In [15]:
sum1=0
for i in final_asst:
    sum1+=len(final_asst[i])
sum1

4516

## Defining dictionaries T_f and T_f_i 

In [16]:
T_f={f:0 for f in range(len(S))}

for f in range(len(S)):
    for v in list(X.index):

        T_f[f]+= x_star[v,f]
        
T_f_i = { (f,i) : 0 for f in range(len(S)) for i in list(C_married.keys())}

T_f_j = { (f,j) : 0 for f in range(len(S)) for j in list(C_ed.keys())}


C_married={i:[] for i in set(X['marital'].values)}

for i in range(len(X['marital'])) :
    C_married[X['marital'].iat[i]].append(X.index[i]) 

for f in range(len(S)):
    for i in list(C_married.keys()):
        for j in C_married[i]:
            T_f_i[f,i]+= x_star[j,f]
            
C_ed={j:[] for j in set(X['default'].values)}

for j in range(len(X['default'])):
    C_ed[X['default'].iat[j]].append(X.index[j]) 

for f in range(len(S)):
    for i in list(C_ed.keys()):
        for j in C_ed[i]:
            T_f_j[f,i]+= x_star[j,f]
          
# print(sum(T_f_i.values()))
# max(T_f_i.values())

## Data to be clustered for LP2, with x*[v,f]>0

In [17]:
Vanilla_data

,age,balance,duration
558,41,720,651
635,33,280,155
1097,45,2096,127
1440,56,1238,1558
2818,57,25,652


## LP2 solved iteratively, until all variables are assigned to a cluster with fairness considerations

In [18]:
Vanilla_data_temp=Vanilla_data

while(len(Vanilla_data_temp)>0):
    model=Model("LP2")

    x={}
    for v in list(X.index):
        for f in range(len(S)):
            if x_star[v,f]>0:
                x[v,f]=model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1)

    for f in range(len(S)):
        model.addConstr(quicksum(x[v,f] for v in list(X.index) if x_star[v,f]>0) <= np.ceil(T_f[f]))
        model.addConstr(quicksum(x[v,f] for v in list(X.index) if x_star[v,f]>0) >= np.floor(T_f[f]))

    for f in range(len(S)):
        for i in list(C_married.keys()):
            model.addConstr(quicksum(x[v,f] for v in C_married[i] if x_star[v,f]>0) <= np.ceil(T_f_i[f,i]))
            model.addConstr(quicksum(x[v,f] for v in C_married[i] if x_star[v,f]>0) >= np.floor(T_f_i[f,i]))

    for f in range(len(S)):
        for i in list(C_ed.keys()):
            model.addConstr(quicksum(x[v,f] for v in C_ed[i] if x_star[v,f]>0) <= np.ceil(T_f_j[f,i]))
            model.addConstr(quicksum(x[v,f] for v in C_ed[i] if x_star[v,f]>0) >= np.floor(T_f_j[f,i]))

    for v in list(Vanilla_data.index):
        model.addConstr(quicksum(x[v,f] for f in range(len(S)) if x_star[v,f]>0)==1)

    Obj=0
    for v in list(Vanilla_data.index):
        for f in range(len(S)):
            if x_star[v,f]>0:
                V = (np.array(Vanilla_data[Vanilla_data.index==v])[0])
                Obj+=x[v,f]*(np.linalg.norm(V-np.array(S[f]))**2)

    model.setObjective(Obj)
    model.optimize()

    for v in list(Vanilla_data.index):
        for f in range(len(S)):
            if (x_star[v,f]>0) and (x[v,f].X==1):
                Vanilla_data_temp=Vanilla_data_temp.drop(v)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 53 rows, 4526 columns and 27166 nonzeros
Model fingerprint: 0x708e1fe4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+05, 6e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 45 rows and 4518 columns
Presolve time: 0.02s
Presolved: 8 rows, 12 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.4787407e+07   1.000000e+01   0.000000e+00      0s
       7    1.2097197e+08   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.03 seconds
Optimal objective  1.209719676e+08


## Assigning v with x*[v,f] = 1 to final_asst after LP2, the dictionary containing the final fair clustered points 

In [19]:
for v in list(Vanilla_data.index):
    for f in range(len(S)):
        if (x_star[v,f]>0) and (x[v,f].X==1):
            final_asst[f].append(v)

## Sanity check, number of variables in final_asst (should be equal to total number of points)

In [20]:
sum1=0
for i in final_asst:
    sum1+=len(final_asst[i])
sum1

4521

## Objective value of fair clustering problem

In [21]:
sum_obj=0
for i in final_asst:
    for j in final_asst[i]:
        V = (np.array(Vanilla_data_copy[Vanilla_data_copy.index==j])[0])
        sum_obj+= np.linalg.norm(V-np.array(S[i]))**2
print('Objective value of fair clustering =', sum_obj)

Objective value of fair clustering = 7017123676.986678


## Cost of fairness

In [22]:
COF=sum_obj/kmeans.inertia_
print("Cost of fairness:",COF)

Cost of fairness: 1.061229950068598


## (Cluster center, group i) for all points

In [23]:
Cluster_index_marital={(i,j):[] for i in range(len(S)) for j in range((len(C_married)))} #i refers to the cluster center index 
                                                                                 #j refers to the Ci index
for i in final_asst:
    for j in range((len(C_married))):
        for k in final_asst[i]:
            if X['marital'][k]==j:
                Cluster_index_marital[i,j].append(k)
                
Cluster_index_ed={(i,j):[] for i in range(len(S)) for j in range((len(C_ed)))} #i refers to the cluster center index 
                                                                                 #j refers to the Ci index
for i in final_asst:
    for j in range((len(C_ed))):
        for k in final_asst[i]:
            if X['default'][k]==j:
                Cluster_index_ed[i,j].append(k)

## Sanity check (summing up all entries - sums up to total number of data points)

In [24]:
sum1=0
for i in final_asst:
    for j in range((len(C_married))):
        Cluster_index_marital[i,j]=len(Cluster_index_marital[i,j])
        sum1+=Cluster_index_marital[i,j]
print(sum1)
sum2=0
for i in final_asst:
    for j in range((len(C_ed))):
        Cluster_index_ed[i,j]=len(Cluster_index_ed[i,j])
        sum2+=Cluster_index_ed[i,j]
print(sum2)

4521
4521


In [25]:
alpha_married = {0: 0.75, 1: 0.75, 2: 0.75}
beta_married = {0: 0.1, 1: 0.15, 2: 0.15}
alpha_ed = {0: 0.99, 1: 0.99}
beta_ed = {0: 0.05, 1: 0.01}

In [26]:
print(Cluster_index_marital)
print(Cluster_index_ed)

{(0, 0): 446, (0, 1): 2272, (0, 2): 974, (1, 0): 15, (1, 1): 90, (1, 2): 43, (2, 0): 65, (2, 1): 420, (2, 2): 174, (3, 0): 2, (3, 1): 15, (3, 2): 5}
{(0, 0): 3624, (0, 1): 68, (1, 0): 147, (1, 1): 1, (2, 0): 652, (2, 1): 7, (3, 0): 22, (3, 1): 0}


## Checking violating MP and RD constraints, and values of corresponding lambda

In [27]:
#Upper bound for default groups
0.99*(3624+68)
0.99*(147+1) #violates by lambda = 0.48
0.99*(652+7)
0.99*(22+0) #violates by lambda = 0.22

21.78

In [28]:
#Lower bound for default groups
0.05*(3624+68)
0.01*(3624+68)
0.05*(147+1)
0.01*(147+1) #violates by lambda =0.48
0.05*(652+7)
0.01*(652+7)
0.05*(22+0)
0.01*(22+0) #violates by lambda = 0.22

0.22

In [29]:
#Upper bound for married groups
0.75*(446+2272+974)
0.75*(15+90+43)
0.75*(65+420+174)
0.75*(2+15+5)

16.5

In [30]:
#Lower bound for married groups
0.1*(446+2272+974)
0.15*(446+2272+974)
0.1*(15+90+43)
0.15*(15+90+43)
0.1*(65+420+174) #violates by lambda = 0.9
0.15*(65+420+174)
0.1*(2+15+5) #violates by lambda = 0.2
0.15*(2+15+5)

3.3

## Calculating values of r_i and r_i_f for marital and default attributes

In [31]:
r_i_f_marital={(i,j):[] for i in range(len(S)) for j in range((len(C_married)))}
sum_cluster=[0,0,0,0]

for i in range(len(S)):
    for j in range(len(C_married)):
        r_i_f_marital[i,j]=Cluster_index_marital[i,j]
        sum_cluster[i]+=Cluster_index_marital[i,j]

for i in range(len(S)):
    for j in range(len(C_married)):
        r_i_f_marital[i,j]=Cluster_index_marital[i,j]/sum_cluster[i]
print(r_i_f_marital, '\n')

r_i_f_ed= {(i,j):[] for i in range(len(S)) for j in range((len(C_ed)))}
sum_cluster=[0,0,0,0]

for i in range(len(S)):
    for j in range(len(C_ed)):
        r_i_f_ed[i,j]=Cluster_index_ed[i,j]
        sum_cluster[i]+=Cluster_index_ed[i,j]

for i in range(len(S)):
    for j in range(len(C_ed)):
        r_i_f_ed[i,j]=Cluster_index_ed[i,j]/sum_cluster[i]
print(r_i_f_ed)

{(0, 0): 0.12080173347778982, (0, 1): 0.6153846153846154, (0, 2): 0.2638136511375948, (1, 0): 0.10135135135135136, (1, 1): 0.6081081081081081, (1, 2): 0.2905405405405405, (2, 0): 0.09863429438543247, (2, 1): 0.637329286798179, (2, 2): 0.26403641881638845, (3, 0): 0.09090909090909091, (3, 1): 0.6818181818181818, (3, 2): 0.22727272727272727} 

{(0, 0): 0.9815817984832069, (0, 1): 0.018418201516793065, (1, 0): 0.9932432432432432, (1, 1): 0.006756756756756757, (2, 0): 0.9893778452200304, (2, 1): 0.010622154779969651, (3, 0): 1.0, (3, 1): 0.0}


In [32]:
r_married=[0,0,0]
for i in C_married:
    r_married[i]=len(C_married[i])/len(X)
print(r_married)

r_ed=[0,0]
for i in C_ed:
    r_ed[i]=len(C_ed[i])/len(X)
print(r_ed)

[0.11678832116788321, 0.6186684361866843, 0.26454324264543244]
[0.9831895598318956, 0.0168104401681044]


## Calculating balance values for each center f and each attribute 

In [36]:
balance_married = [0]*len(S)
for i in range(len(S)):
    balance_married[i]=[]
    for j in range(len(C_married)):
        if (r_i_f_marital[i,j]!=0) and r_married[j]!=0:
            balance_married[i].append(r_i_f_marital[i,j]/r_married[j])
            balance_married[i].append(r_married[j]/r_i_f_marital[i,j])
    balance_married[i]=min(balance_married[i])
print('Balance values for attribute = marital:',balance_married)

balance_ed = [0]*len(S)
for i in range(len(S)):
    balance_ed[i]=[]
    for j in range(len(C_ed)):
        if (r_i_f_ed[i,j]!=0) and r_ed[j]!=0:
            balance_ed[i].append(r_i_f_ed[i,j]/r_ed[j])
            balance_ed[i].append(r_ed[j]/r_i_f_ed[i,j])
    balance_ed[i]=min(balance_ed[i])
print('Balance values for attribute = default:',balance_ed)

Balance values for attribute = marital: [0.9667768649144054, 0.867820945945946, 0.8445561456752656, 0.7784090909090909]
Balance values for attribute = default: [0.9127080161859037, 0.4019381223328592, 0.631878444213721, 0.9831895598318956]
